## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Sao Jose Da Coroa Grande,-8.8978,-35.1478,73.13,88,68,9.42,BR,broken clouds
1,1,Saint-Philippe,-21.3585,55.7679,67.59,73,0,3.24,RE,clear sky
2,2,Mar Del Plata,-38.0023,-57.5575,51.87,82,0,3.44,AR,clear sky
3,3,Ushuaia,-54.8000,-68.3000,47.86,57,20,16.11,AR,few clouds
4,4,Adrar,20.5022,-10.0711,85.80,36,54,12.93,MR,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Sao Jose Da Coroa Grande,-8.8978,-35.1478,73.13,88,68,9.42,BR,broken clouds
1,1,Saint-Philippe,-21.3585,55.7679,67.59,73,0,3.24,RE,clear sky
4,4,Adrar,20.5022,-10.0711,85.80,36,54,12.93,MR,broken clouds
5,5,Sao Filipe,14.8961,-24.4956,75.69,85,58,7.38,CV,light rain
9,9,Port Blair,11.6667,92.7500,80.64,89,40,2.30,IN,scattered clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                388
City                   388
Lat                    388
Lng                    388
Max Temp               388
Humidity               388
Cloudiness             388
Wind Speed             388
Country                388
Current Description    388
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_df = preferred_cities_df.dropna(axis="index", how="any")
clean_preferred_df.count()

City_ID                388
City                   388
Lat                    388
Lng                    388
Max Temp               388
Humidity               388
Cloudiness             388
Wind Speed             388
Country                388
Current Description    388
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Jose Da Coroa Grande,BR,73.13,broken clouds,-8.8978,-35.1478,
1,Saint-Philippe,RE,67.59,clear sky,-21.3585,55.7679,
4,Adrar,MR,85.80,broken clouds,20.5022,-10.0711,
5,Sao Filipe,CV,75.69,light rain,14.8961,-24.4956,
9,Port Blair,IN,80.64,scattered clouds,11.6667,92.7500,
10,Nouadhibou,MR,70.30,scattered clouds,20.9310,-17.0347,
12,Boueni,YT,75.18,clear sky,-12.9025,45.0761,
14,Jamestown,US,74.79,broken clouds,42.0970,-79.2353,
15,Tukrah,LY,78.08,clear sky,32.5341,20.5791,
17,Bandarbeyla,SO,79.00,scattered clouds,9.4942,50.8122,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(axis="index", how="any")
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Jose Da Coroa Grande,BR,73.13,broken clouds,-8.8978,-35.1478,Crôa Mares Hotel
1,Saint-Philippe,RE,67.59,clear sky,-21.3585,55.7679,Le four à pain
4,Adrar,MR,85.80,broken clouds,20.5022,-10.0711,
5,Sao Filipe,CV,75.69,light rain,14.8961,-24.4956,Hotel Xaguate
9,Port Blair,IN,80.64,scattered clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))